# Import Libraries

Here, we import required libraries. The purpose of each package is as follows.

- glob: to find the data sets
- pandas: to parse csv
- matplotlib: to read ppm file
- cv2: to resize the read images
- os: general operations on path (e.g., os.path.dirname)
- numpy: general numerical operation
- sklearn: to split test data
- tensorflow: CNN

In [40]:
import glob
import pandas as pd
# import matplotlib
import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.contrib.layers import flatten
# from pipeline import NewralNetwork, make_adam, Session, build_pipeline

# Read Data for Training

In [66]:
TRAIN_IMAGE_DIR = 'data/GTSRB_Final_Training_Images/GTSRB/Final_Training/Images/'

data_files = []
for train_file in glob.glob(os.path.join(TRAIN_IMAGE_DIR, '*/GT-*.csv')):
    folder = os.path.dirname(train_file)
    df = pd.read_csv(train_file, sep=';')
    df['Filename'] = df['Filename'].apply(lambda x: os.path.join(folder, x))
    data_files.append(df)

In [67]:
train_df = pd.concat(data_files, ignore_index=True)
train_df.head()

,Filename,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId
0,data/GTSRB_Final_Training_Images/GTSRB/Final_T...,38,35,5,5,33,30,31
1,data/GTSRB_Final_Training_Images/GTSRB/Final_T...,40,36,6,5,34,31,31
2,data/GTSRB_Final_Training_Images/GTSRB/Final_T...,40,37,5,5,34,32,31
3,data/GTSRB_Final_Training_Images/GTSRB/Final_T...,41,39,6,6,36,34,31
4,data/GTSRB_Final_Training_Images/GTSRB/Final_T...,43,37,6,5,38,32,31


In [84]:
def load_image(image_file):
    """
    Read image file into numpy array (RGB)
    """
    return plt.imread(image_file)

In [145]:
INPUT_SHAPE = (32, 32, 3)

def resize_image(image, shape=INPUT_SHAPE[:2]):
    return cv2.resize(image, shape)

loader = lambda image_file: resize_image(load_image(image_file))
load_all = np.vectorize(loader)

# Load The Data

In [156]:
X = train_df['Filename'].values
y = train_df['ClassId'].values

train_data_filenames, eval_data_filenames, train_labels, eval_labels = train_test_split(X, y, stratify=y, test_size=8000, random_state=0)
train_data = []
for f in train_data_filenames:
    train_data.append(loader(f))
train_data = np.array(train_data, dtype=float)
eval_data = []
for f in eval_data_filenames:
    eval_data.append(loader(f))
eval_data = np.array(eval_data, dtype=float) 
print(len(train_data))
print(len(eval_data))

31209
8000


In [142]:
#np.unique(train_data)
train_data.head

AttributeError: 'list' object has no attribute 'head'

# Main Function

## Training of the model

In [160]:
gtsrb_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/gtsrb_convnet_model")

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
gtsrb_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook])


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/gtsrb_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x127e1f518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/gtsrb_convnet_model/model.ckpt.
INFO:tensorflow

INFO:tensorflow:global_step/sec: 1.25799
INFO:tensorflow:probabilities = [[0.00961209 0.04275379 0.0568032  ... 0.01836062 0.01347492 0.01399498]
 [0.0082774  0.03469462 0.04543413 ... 0.01238074 0.01268274 0.00858661]
 [0.01878978 0.0255833  0.03300976 ... 0.01638716 0.01797376 0.01412671]
 ...
 [0.00596292 0.0508219  0.13322646 ... 0.02446701 0.0029958  0.00246199]
 [0.01817645 0.03458592 0.02957172 ... 0.02183006 0.01362973 0.01314261]
 [0.01761162 0.03072088 0.04280914 ... 0.02221676 0.0151155  0.01443174]] (40.712 sec)
INFO:tensorflow:loss = 3.5823607444763184, step = 701 (79.493 sec)
INFO:tensorflow:probabilities = [[0.01945578 0.03432644 0.02721125 ... 0.01913492 0.01645483 0.02078906]
 [0.01717967 0.03273084 0.0338758  ... 0.02688779 0.01632479 0.02206004]
 [0.02113716 0.02580692 0.02598742 ... 0.02179666 0.01992078 0.0220049 ]
 ...
 [0.01223096 0.02924758 0.04676444 ... 0.01314796 0.00754462 0.01197284]
 [0.00421732 0.03559675 0.12075649 ... 0.01050255 0.01049263 0.01893152]
 

INFO:tensorflow:loss = 2.994582414627075, step = 1401 (68.466 sec)
INFO:tensorflow:probabilities = [[0.01036878 0.01922297 0.02754498 ... 0.00258907 0.00147694 0.00109426]
 [0.00003073 0.00334351 0.02557221 ... 0.00812067 0.00000541 0.00083314]
 [0.00222609 0.04728179 0.0074796  ... 0.01725038 0.00718105 0.00441074]
 ...
 [0.00055006 0.00963961 0.05337203 ... 0.00144965 0.000425   0.00273349]
 [0.000252   0.00033585 0.00161629 ... 0.00039157 0.00000053 0.00012339]
 [0.00062202 0.02131969 0.0064325  ... 0.02042961 0.00031041 0.02526368]] (34.463 sec)
INFO:tensorflow:global_step/sec: 1.40181
INFO:tensorflow:probabilities = [[0.00246184 0.00727831 0.05948115 ... 0.00377686 0.00017572 0.00033985]
 [0.02795985 0.03704755 0.04755696 ... 0.01626081 0.00969761 0.01258713]
 [0.01248812 0.02310235 0.03164857 ... 0.01839068 0.00969613 0.01495441]
 ...
 [0.00007112 0.10397384 0.01230916 ... 0.00000793 0.         0.00000054]
 [0.00055334 0.00812384 0.01079362 ... 0.00457968 0.00018939 0.00009952]
 

INFO:tensorflow:global_step/sec: 0.97316
INFO:tensorflow:probabilities = [[0.01989972 0.01479577 0.01143933 ... 0.01010549 0.01895922 0.0085546 ]
 [0.01210953 0.0063066  0.00501952 ... 0.02592237 0.00509416 0.00589825]
 [0.00000715 0.01734089 0.49074105 ... 0.00000094 0.         0.00000001]
 ...
 [0.00025989 0.00633796 0.01008764 ... 0.00032875 0.00110175 0.00083467]
 [0.00000006 0.00010425 0.00031432 ... 0.00247898 0.         0.        ]
 [0.00001878 0.00117153 0.00086815 ... 0.00249017 0.00018847 0.0000316 ]] (44.292 sec)
INFO:tensorflow:loss = 1.4133938550949097, step = 2201 (102.755 sec)
INFO:tensorflow:probabilities = [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.00000001 0.00000053 ... 0.00006099 0.         0.        ]
 ...
 [0.00000647 0.00008302 0.00001902 ... 0.00001934 0.00002075 0.00001448]
 [0.0001385  0.00119757 0.00085553 ... 0.0005602  0.00000331 0.00021815]

INFO:tensorflow:loss = 1.093658208847046, step = 2901 (84.691 sec)
INFO:tensorflow:probabilities = [[0.00001318 0.84465425 0.01910151 ... 0.00000729 0.00032224 0.00000199]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.00776486 0.0262236  0.00362343 ... 0.01497539 0.00088951 0.00324381]
 ...
 [0.0013701  0.64931459 0.25941152 ... 0.00015407 0.00010749 0.00034806]
 [0.00216028 0.20196627 0.01003216 ... 0.00002806 0.00000235 0.00001927]
 [0.03348145 0.18163224 0.01523852 ... 0.00068322 0.00360961 0.00780862]] (39.219 sec)
INFO:tensorflow:global_step/sec: 1.32036
INFO:tensorflow:probabilities = [[0.00000001 0.         0.02052092 ... 0.00000015 0.00000023 0.        ]
 [0.00001772 0.00019098 0.00002888 ... 0.00000285 0.0000024  0.00005346]
 [0.0002859  0.03089859 0.00463442 ... 0.00000022 0.00000054 0.0000047 ]
 ...
 [0.00000009 0.00000044 0.00000025 ... 0.00000123 0.00003103 0.00000058]
 [0.00082307 0.00549558 0.00519261 ... 0.00003811 0.00000966 0.00002513]
 

...

INFO:tensorflow:loss = 0.06573336571455002, step = 17201 (75.264 sec)
INFO:tensorflow:probabilities = [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.00000036 0.         ... 0.         0.00000007 0.00017676]
 [0.         0.         0.         ... 0.00000003 0.00000008 0.00000031]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]] (38.261 sec)
INFO:tensorflow:global_step/sec: 1.31097
INFO:tensorflow:probabilities = [[0.0000036  0.00000025 0.00266894 ... 0.00000139 0.00000007 0.01118315]
 [0.00000107 0.00001328 0.00000412 ... 0.00000464 0.00000817 0.00000012]
 [0.00003595 0.00001356 0.00099642 ... 0.00014451 0.00005947 0.00016857]
 ...
 [0.         0.00000002 0.         ... 0.         0.99994008 0.00003813]
 [0.         0.         0.         ... 0.         0.         0.        

INFO:tensorflow:global_step/sec: 1.32702
INFO:tensorflow:probabilities = [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.00000012 0.00000001 0.         ... 0.00000007 0.00000024 0.00000003]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.00000003 0.00002023 0.00000074 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.0000004  0.95166488 ... 0.         0.         0.        ]] (37.413 sec)
INFO:tensorflow:loss = 0.1177186444401741, step = 18001 (75.355 sec)
INFO:tensorflow:probabilities = [[0.00000159 0.0000235  0.00051551 ... 0.00000001 0.         0.00000001]
 [0.00053344 0.00002826 0.00588954 ... 0.00000287 0.0000568  0.00007895]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.00000001 ... 0.         0.         0.        ]
 [1.         0.         0.         ... 0.         0.         0.        ]

INFO:tensorflow:global_step/sec: 1.30299
INFO:tensorflow:probabilities = [[0.51719995 0.48279949 0.         ... 0.         0.         0.        ]
 [0.0000775  0.00087013 0.24132604 ... 0.00000009 0.00000002 0.00000011]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.000001   ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]] (38.588 sec)
INFO:tensorflow:loss = 0.04542580991983414, step = 18801 (76.745 sec)
INFO:tensorflow:probabilities = [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.00000011 0.         ... 0.         0.         0.        ]
 [0.         0.00000002 0.00000071 ... 0.         0.         0.        ]
 ...
 [0.         0.99685668 0.00002224 ... 0.         0.         0.        ]
 [0.         0.0001634  0.00006608 ... 0.         0.         0.        

INFO:tensorflow:loss = 0.03267683833837509, step = 19501 (75.235 sec)
INFO:tensorflow:probabilities = [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.00000105 0.00000077 0.00021807 ... 0.00003834 0.00004103 0.00000126]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.00005585 0.00000159 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]] (38.093 sec)
INFO:tensorflow:global_step/sec: 1.31087
INFO:tensorflow:probabilities = [[0.00000048 0.00000499 0.00006382 ... 0.00000405 0.00000614 0.00008864]
 [0.         0.         0.         ... 0.00000027 0.         0.        ]
 [0.00010242 0.0106298  0.00750326 ... 0.00414879 0.00224553 0.00058105]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.00000005 0.00003332 0.00012624 ... 0.         0.         0.00000001

NameError: name 'mnist_classifier' is not defined

## Evaluation of the learned

In [161]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = gtsrb_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-16-02:24:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/gtsrb_convnet_model/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-16-02:24:36
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9775, global_step = 20000, loss = 0.10542794
{'accuracy': 0.9775, 'loss': 0.10542794, 'global_step': 20000}


In [ ]:
if __name__ == "__main__":
  tf.app.run()

# CNN Model

In [159]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1] + list(INPUT_SHAPE))

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=43)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)